#Мини-игра «Пати против Босса»

##Импорт библиотек

In [6]:
import random
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Tuple, Iterator

##Базовые классы и дескрипторы
Содержит:

1. BoundedStat - дескриптор для ограничения значений характеристик (HP, MP не могут быть отрицательными)

2. Human - базовый класс всех существ с основными характеристиками и методами

3. Character - абстрактный класс персонажей с навыками и перезарядкой

In [7]:
class BoundedStat:
    """Дескриптор для валидации характеристик"""
    def __init__(self, min_val: int = 0, max_val: int = 100):
        self.min_val = min_val
        self.max_val = max_val

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name, 0)

    def __set__(self, instance, value):
        value = max(self.min_val, min(value, self.max_val))
        instance.__dict__[self.name] = value

class Human:
    """Базовый класс для всех существ"""
    hp = BoundedStat(0, 1000)
    mp = BoundedStat(0, 500)
    strength = BoundedStat(1, 100)
    agility = BoundedStat(1, 100)
    intelligence = BoundedStat(1, 100)

    def __init__(self, name: str, level: int = 1):
        self.name = name
        self.level = level
        self._max_hp = 100
        self._max_mp = 50
        self._hp = 100
        self._mp = 50
        self._strength = 10
        self._agility = 10
        self._intelligence = 10
        self.effects = []

    @property
    def is_alive(self) -> bool:
        return self.hp > 0

    def take_damage(self, damage: int):
        # Проверяем щиты перед нанесением урона
        shield = next((eff for eff in self.effects if eff.name == "Щит"), None)
        if shield and hasattr(shield, 'current_shield'):
            if shield.current_shield >= damage:
                shield.current_shield -= damage
                damage = 0
            else:
                damage -= shield.current_shield
                shield.current_shield = 0
                self.effects.remove(shield)

        self.hp -= damage
        return damage

    def heal(self, amount: int):
        old_hp = self.hp
        self.hp += amount
        return self.hp - old_hp

    def add_effect(self, effect):
        self.effects.append(effect)
        effect.on_apply(self)

    def remove_effect(self, effect):
        if effect in self.effects:
            self.effects.remove(effect)
            effect.on_remove(self)

    def process_effects(self):
        for effect in self.effects[:]:
            effect.on_turn(self)
            if effect.duration <= 0:
                self.remove_effect(effect)

    def __str__(self):
        return f"{self.name} (HP: {self.hp}/{self._max_hp}, MP: {self.mp}/{self._max_mp})"

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', level={self.level})"

class Character(Human, ABC):
    """Абстрактный класс персонажа"""
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.skills_cooldown = {}

    @abstractmethod
    def basic_attack(self, target: Human) -> str:
        pass

    @abstractmethod
    def use_skill(self, target: Human, skill_name: str) -> str:
        pass

    def update_cooldowns(self):
        for skill in list(self.skills_cooldown.keys()):
            self.skills_cooldown[skill] = max(0, self.skills_cooldown[skill] - 1)

    def can_use_skill(self, skill_name: str) -> bool:
        return self.skills_cooldown.get(skill_name, 0) == 0

##Классы персонажей и миксины
Содержит:

1. CritMixin - миксин для критического урона

2. Warrior - класс воина (сила, высокая HP, физические атаки)

3. Mage - класс мага (интеллект, магические атаки, эффекты)

4. Healer - класс лекаря (лечение, очистка эффектов, бафы)

In [8]:
# Миксины и классы персонажей
class CritMixin:
    """Миксин для критического урона"""
    def calculate_crit(self, base_damage: int, crit_chance: float = 0.1) -> Tuple[int, bool]:
        if random.random() < crit_chance:
            return int(base_damage * 1.5), True
        return base_damage, False

class Warrior(Character, CritMixin):
    """Класс воина"""
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._max_hp = 150 + level * 20
        self._max_mp = 30 + level * 5
        self.hp = self._max_hp
        self.mp = self._max_mp
        self.strength = 20 + level * 3
        self.agility = 15 + level * 2
        self.intelligence = 8 + level * 1

        self.skills = {
            "power_strike": {"mp_cost": 15, "cooldown": 2},
            "whirlwind": {"mp_cost": 25, "cooldown": 3},
            "taunt": {"mp_cost": 10, "cooldown": 4}
        }

    def basic_attack(self, target: Human) -> str:
        damage, is_crit = self.calculate_crit(self.strength)
        actual_damage = target.take_damage(damage)
        crit_text = " КРИТИЧЕСКИЙ УРОН!" if is_crit else ""
        return f"{self.name} атакует {target.name} и наносит {actual_damage} урона!{crit_text}"

    def use_skill(self, target: Human, skill_name: str) -> str:
        if not self.can_use_skill(skill_name):
            return f"Навык {skill_name} на перезарядке!"

        skill = self.skills.get(skill_name)
        if not skill:
            return f"Неизвестный навык: {skill_name}"

        if self.mp < skill["mp_cost"]:
            return f"Недостаточно маны для {skill_name}!"

        self.mp -= skill["mp_cost"]
        self.skills_cooldown[skill_name] = skill["cooldown"]

        if skill_name == "power_strike":
            damage = int(self.strength * 1.8)
            actual_damage = target.take_damage(damage)
            return f"{self.name} использует Мощный удар на {target.name} и наносит {actual_damage} урона!"

        elif skill_name == "whirlwind":
            damage = int(self.strength * 1.2)
            actual_damage = target.take_damage(damage)
            return f"{self.name} использует Вихрь и наносит {actual_damage} урона {target.name}!"

        elif skill_name == "taunt":
            target.add_effect(TauntEffect(3, self))
            return f"{self.name} провоцирует {target.name}!"

        return "Неизвестный навык"

class Mage(Character):
    """Класс мага"""
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._max_hp = 80 + level * 10
        self._max_mp = 100 + level * 15
        self.hp = self._max_hp
        self.mp = self._max_mp
        self.strength = 8 + level * 1
        self.agility = 12 + level * 2
        self.intelligence = 25 + level * 4

        self.skills = {
            "fireball": {"mp_cost": 20, "cooldown": 1},
            "frost_bolt": {"mp_cost": 15, "cooldown": 2},
            "arcane_shield": {"mp_cost": 30, "cooldown": 4}
        }

    def basic_attack(self, target: Human) -> str:
        damage = self.intelligence // 2
        actual_damage = target.take_damage(damage)
        return f"{self.name} атакует посохом {target.name} и наносит {actual_damage} урона!"

    def use_skill(self, target: Human, skill_name: str) -> str:
        if not self.can_use_skill(skill_name):
            return f"Навык {skill_name} на перезарядке!"

        skill = self.skills.get(skill_name)
        if not skill:
            return f"Неизвестный навык: {skill_name}"

        if self.mp < skill["mp_cost"]:
            return f"Недостаточно маны для {skill_name}!"

        self.mp -= skill["mp_cost"]
        self.skills_cooldown[skill_name] = skill["cooldown"]

        if skill_name == "fireball":
            damage = int(self.intelligence * 1.5)
            actual_damage = target.take_damage(damage)
            target.add_effect(BurnEffect(2, damage // 4))
            return f"{self.name} бросает Огненный шар в {target.name} и наносит {actual_damage} урона!"

        elif skill_name == "frost_bolt":
            damage = int(self.intelligence * 1.2)
            actual_damage = target.take_damage(damage)
            target.add_effect(SlowEffect(2))
            return f"{self.name} запускает Ледяную стрелу в {target.name} и наносит {actual_damage} урона!"

        elif skill_name == "arcane_shield":
            self.add_effect(ShieldEffect(3, self.intelligence))
            return f"{self.name} создает Магический щит!"

        return "Неизвестный навык"

class Healer(Character):
    """Класс лекаря"""
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._max_hp = 100 + level * 12
        self._max_mp = 80 + level * 12
        self.hp = self._max_hp
        self.mp = self._max_mp
        self.strength = 10 + level * 1
        self.agility = 14 + level * 2
        self.intelligence = 20 + level * 3

        self.skills = {
            "heal": {"mp_cost": 20, "cooldown": 2},
            "purify": {"mp_cost": 15, "cooldown": 3},
            "blessing": {"mp_cost": 25, "cooldown": 4}
        }

    def basic_attack(self, target: Human) -> str:
        damage = self.strength // 2 + self.intelligence // 4
        actual_damage = target.take_damage(damage)
        return f"{self.name} атакует {target.name} и наносит {actual_damage} урона!"

    def use_skill(self, target: Human, skill_name: str) -> str:
        if not self.can_use_skill(skill_name):
            return f"Навык {skill_name} на перезарядке!"

        skill = self.skills.get(skill_name)
        if not skill:
            return f"Неизвестный навык: {skill_name}"

        if self.mp < skill["mp_cost"]:
            return f"Недостаточно маны для {skill_name}!"

        self.mp -= skill["mp_cost"]
        self.skills_cooldown[skill_name] = skill["cooldown"]

        if skill_name == "heal":
            heal_amount = int(self.intelligence * 1.5)
            actual_heal = target.heal(heal_amount)
            return f"{self.name} лечит {target.name} на {actual_heal} HP!"

        elif skill_name == "purify":
            removed_effects = []
            for effect in target.effects[:]:
                if not effect.is_positive:
                    target.remove_effect(effect)
                    removed_effects.append(effect.name)

            if removed_effects:
                return f"{self.name} очищает {target.name} от эффектов: {', '.join(removed_effects)}"
            else:
                return f"{self.name} пытается очистить {target.name}, но не находит негативных эффектов!"

        elif skill_name == "blessing":
            target.add_effect(BlessEffect(3, self.intelligence // 2))
            return f"{self.name} благословляет {target.name}!"

        return "Неизвестный навык"

##Система эффектов
Содержит систему временных эффектов:

1. Горение - урон по времени

2. Замедление - снижение ловкости

3. Щит - поглощение урона

4. Благословение - увеличение характеристик

5. Провокация - принуждение к атаке

Механика: Каждый эффект имеет длительность и срабатывает каждый ход

In [9]:
# Система эффектов
class Effect(ABC):
    """Базовый класс эффектов"""
    def __init__(self, name: str, duration: int, is_positive: bool = False):
        self.name = name
        self.duration = duration
        self.is_positive = is_positive

    @abstractmethod
    def on_apply(self, target: Human):
        pass

    @abstractmethod
    def on_turn(self, target: Human):
        pass

    @abstractmethod
    def on_remove(self, target: Human):
        pass

class BurnEffect(Effect):
    """Эффект горения"""
    def __init__(self, duration: int, damage_per_turn: int):
        super().__init__("Горение", duration, False)
        self.damage_per_turn = damage_per_turn

    def on_apply(self, target: Human):
        print(f"🔥 {target.name} начинает гореть!")

    def on_turn(self, target: Human):
        damage = target.take_damage(self.damage_per_turn)
        print(f"🔥 {target.name} получает {damage} урона от горения!")
        self.duration -= 1

    def on_remove(self, target: Human):
        print(f"✅ {target.name} перестал гореть.")

class SlowEffect(Effect):
    """Эффект замедления"""
    def __init__(self, duration: int):
        super().__init__("Замедление", duration, False)
        self.original_agility = 0

    def on_apply(self, target: Human):
        self.original_agility = target.agility
        target.agility = max(1, target.agility // 2)
        print(f"🐌 {target.name} замедлен!")

    def on_turn(self, target: Human):
        self.duration -= 1

    def on_remove(self, target: Human):
        target.agility = self.original_agility
        print(f"✅ {target.name} больше не замедлен.")

class ShieldEffect(Effect):
    """Эффект щита"""
    def __init__(self, duration: int, shield_power: int):
        super().__init__("Щит", duration, True)
        self.shield_power = shield_power
        self.current_shield = shield_power

    def on_apply(self, target: Human):
        print(f"🛡️ {target.name} получает щит на {self.shield_power} урона!")

    def on_turn(self, target: Human):
        self.duration -= 1

    def on_remove(self, target: Human):
        print(f"🛡️ Щит {target.name} исчезает.")

class BlessEffect(Effect):
    """Эффект благословения"""
    def __init__(self, duration: int, bonus: int):
        super().__init__("Благословение", duration, True)
        self.bonus = bonus
        self.original_stats = {}

    def on_apply(self, target: Human):
        self.original_stats['strength'] = target.strength
        self.original_stats['intelligence'] = target.intelligence
        target.strength += self.bonus
        target.intelligence += self.bonus
        print(f"✨ {target.name} получает благословение!")

    def on_turn(self, target: Human):
        self.duration -= 1

    def on_remove(self, target: Human):
        target.strength = self.original_stats['strength']
        target.intelligence = self.original_stats['intelligence']
        print(f"✨ Благословение {target.name} заканчивается.")

class TauntEffect(Effect):
    """Эффект провокации"""
    def __init__(self, duration: int, taunter: Human):
        super().__init__("Провокация", duration, False)
        self.taunter = taunter

    def on_apply(self, target: Human):
        print(f"🎯 {target.name} спровоцирован {self.taunter.name}!")

    def on_turn(self, target: Human):
        self.duration -= 1

    def on_remove(self, target: Human):
        print(f"✅ {target.name} больше не спровоцирован.")

##Босс и стратегии
Реализует паттерн Strategy для босса:

1. Агрессивная стратегия - мощные атаки, огненные взрывы

2. Оборонительная стратегия - лечение, щиты, замедление врагов

3. Босс - автоматически меняет стратегию при потере HP

Фазы боя:

1. Выше 60% HP → агрессивная тактика

2. Ниже 60% HP → оборонительная тактика

In [10]:
# Босс и стратегии поведения
class BossStrategy(ABC):
    """Стратегия поведения босса"""
    @abstractmethod
    def execute(self, boss, targets: List[Human]) -> str:
        pass

class AggressiveStrategy(BossStrategy):
    """Агрессивная стратегия"""
    def execute(self, boss, targets: List[Human]) -> str:
        if boss.mp >= 30 and random.random() < 0.6:
            boss.mp -= 30
            result = f"🐉 {boss.name} использует Огненный взрыв!\n"
            total_damage = 0
            for target in targets:
                if target.is_alive:
                    damage = target.take_damage(boss.strength * 2)
                    total_damage += damage
                    target.add_effect(BurnEffect(2, boss.strength // 3))
                    result += f"  🔥 {target.name} получает {damage} урона и начинает гореть!\n"
            return result
        else:
            alive_targets = [t for t in targets if t.is_alive]
            if not alive_targets:
                return "Нет целей для атаки"
            target = min(alive_targets, key=lambda x: x.hp)
            damage = target.take_damage(boss.strength)
            return f"🐉 {boss.name} яростно атакует {target.name} и наносит {damage} урона!"

class DefensiveStrategy(BossStrategy):
    """Оборонительная стратегия"""
    def execute(self, boss, targets: List[Human]) -> str:
        if boss.hp < boss._max_hp * 0.3 and boss.mp >= 20:
            boss.mp -= 20
            heal_amount = boss.heal(boss.strength * 3)
            return f"🐉 {boss.name} сосредотачивается и восстанавливает {heal_amount} HP!"
        elif boss.mp >= 25:
            boss.mp -= 25
            result = f"🐉 {boss.name} создает защитное поле!\n"
            for target in targets:
                if target.is_alive:
                    target.add_effect(SlowEffect(3))
                    result += f"  🐌 {target.name} замедлен!\n"
            boss.add_effect(ShieldEffect(2, boss.strength * 2))
            result += f"  🛡️ {boss.name} получает щит!"
            return result
        else:
            alive_targets = [t for t in targets if t.is_alive]
            if not alive_targets:
                return "Нет целей для атаки"
            target = random.choice(alive_targets)
            damage = target.take_damage(boss.strength)
            return f"🐉 {boss.name} атакует {target.name} и наносит {damage} урона!"

class Boss(Human):
    """Класс босса с фазами"""
    def __init__(self, name: str, level: int = 10):
        super().__init__(name, level)
        self._max_hp = 500 + level * 50
        self._max_mp = 200 + level * 20
        self.hp = self._max_hp
        self.mp = self._max_mp
        self.strength = 30 + level * 3
        self.agility = 20 + level * 2
        self.intelligence = 25 + level * 2

        self.strategies = {
            "aggressive": AggressiveStrategy(),
            "defensive": DefensiveStrategy()
        }
        self.current_strategy = "aggressive"

    @property
    def phase(self) -> str:
        if self.hp > self._max_hp * 0.6:
            return "aggressive"
        else:
            return "defensive"

    def take_turn(self, targets: List[Human]) -> str:
        new_strategy = self.phase
        if new_strategy != self.current_strategy:
            self.current_strategy = new_strategy
            strategy_name = "Агрессивную" if new_strategy == "aggressive" else "Оборонительную"
            print(f"⚡ {self.name} меняет тактику на {strategy_name}!")

        strategy = self.strategies[self.current_strategy]
        return strategy.execute(self, targets)

    def basic_attack(self, target: Human) -> str:
        damage = self.strength
        actual_damage = target.take_damage(damage)
        return f"{self.name} атакует {target.name} и наносит {actual_damage} урона!"

## Система боя
Организует игровой процесс:

1. TurnOrder - итератор для порядка ходов (по ловкости)

2. LoggerMixin - логирование состояния боя

3. Battle - основной игровой цикл

Функциональность:

1. Управление очередностью ходов

2. Обработка эффектов каждый раунд

3. Проверка условий победы/поражения

4. Интерфейс выбора действий для игрока

In [11]:
# Система боя и порядок ходов
class TurnOrder:
    """Итератор для определения порядка ходов"""
    def __init__(self, participants: List[Human]):
        self.participants = participants
        self.current_turn = 0

    def __iter__(self) -> Iterator[Human]:
        self.participants.sort(key=lambda x: x.agility, reverse=True)
        self.current_turn = 0
        return self

    def __next__(self) -> Human:
        if self.current_turn >= len(self.participants):
            raise StopIteration

        participant = self.participants[self.current_turn]
        self.current_turn += 1
        return participant

class LoggerMixin:
    """Миксин для логирования"""
    def log_round_start(self, round_num: int):
        print(f"\n{'='*50}")
        print(f"🎯 РАУНД {round_num}")
        print(f"{'='*50}")

    def log_character_status(self, characters: List[Human]):
        print("\n📊 Текущее состояние:")
        for char in characters:
            status = "✅" if char.is_alive else "💀"
            effects = ", ".join([eff.name for eff in char.effects]) if char.effects else "нет"
            print(f"  {status} {char} | Эффекты: {effects}")

class Battle(LoggerMixin):
    """Основной класс битвы"""
    def __init__(self, party: List[Character], boss: Boss, seed: int = None):
        if seed is not None:
            random.seed(seed)

        self.party = party
        self.boss = boss
        self.round = 0
        self.is_battle_over = False

    def check_battle_end(self) -> bool:
        party_alive = any(char.is_alive for char in self.party)
        boss_alive = self.boss.is_alive

        if not party_alive:
            print("\n💀 БОСС ПОБЕДИЛ! Все персонажи мертвы.")
            return True
        elif not boss_alive:
            print("\n🎉 ПОБЕДА! Босс повержен!")
            return True

        return False

    def process_effects(self):
        all_participants = self.party + [self.boss]
        for participant in all_participants:
            if participant.is_alive:
                participant.process_effects()

    def run_battle(self):
        print("🎮 НАЧАЛО БИТВЫ!")
        print(f"👥 Партия: {[char.name for char in self.party]}")
        print(f"🐉 Босс: {self.boss.name} (Уровень {self.boss.level})")

        while not self.is_battle_over:
            self.round += 1
            self.log_round_start(self.round)

            self.process_effects()

            all_participants = [p for p in self.party + [self.boss] if p.is_alive]
            turn_order = TurnOrder(all_participants)

            for participant in turn_order:
                if not participant.is_alive:
                    continue

                if self.check_battle_end():
                    self.is_battle_over = True
                    break

                print(f"\n--- Ход {participant.name} ---")

                if isinstance(participant, Boss):
                    result = participant.take_turn(self.party)
                    print(result)
                else:
                    self.player_turn(participant)

            for char in self.party:
                if char.is_alive:
                    char.update_cooldowns()

            self.log_character_status(self.party + [self.boss])

            if self.check_battle_end():
                self.is_battle_over = True
                break

    def player_turn(self, character: Character):
        print(f"1. ⚔️ Базовая атака")
        print(f"2. 🔮 Использовать навык")

        available_skills = []
        if hasattr(character, 'skills'):
            for i, skill_name in enumerate(character.skills.keys(), 3):
                cooldown = character.skills_cooldown.get(skill_name, 0)
                status = "✅" if character.can_use_skill(skill_name) else f"⏳({cooldown})"
                mp_cost = character.skills[skill_name]["mp_cost"]
                print(f"{i}. {skill_name} ({mp_cost} MP) {status}")
                available_skills.append(skill_name)

        try:
            choice = int(input("Выберите действие: "))

            if choice == 1:
                target = self.select_target()
                if target:
                    result = character.basic_attack(target)
                    print(result)

            elif choice == 2:
                if not available_skills:
                    print("Нет доступных навыков!")
                    return

                skill_choice = int(input("Выберите навык: ")) - 3
                if 0 <= skill_choice < len(available_skills):
                    skill_name = available_skills[skill_choice]
                    target = self.select_target()
                    if target:
                        result = character.use_skill(target, skill_name)
                        print(result)
                else:
                    print("Неверный выбор навыка!")

            else:
                print("Неверный выбор!")

        except ValueError:
            print("Пожалуйста, введите число!")

    def select_target(self) -> Optional[Human]:
        print("\n🎯 Выберите цель:")
        print(f"1. {self.boss.name} (HP: {self.boss.hp})")

        for i, ally in enumerate(self.party, 2):
            status = "✅" if ally.is_alive else "💀"
            print(f"{i}. {ally.name} {status} (HP: {ally.hp})")

        try:
            choice = int(input("Выберите цель: "))
            if choice == 1:
                return self.boss
            elif 2 <= choice <= len(self.party) + 1:
                target = self.party[choice - 2]
                if target.is_alive:
                    return target
                else:
                    print("Цель мертва!")
                    return None
            else:
                print("Неверный выбор!")
                return None
        except ValueError:
            print("Пожалуйста, введите число!")
            return None

##Основная игра и запуск
Финальная ячейка:

1. Game - основной класс управления игрой

2. Создание партии персонажей

3. Настройка сложности босса

4. Запуск игрового цикла

Процесс игры:

1. Создание 3 персонажей

2. Выбор сложности босса

3. Настройка случайной генерации (seed)

4. Запуск пошаговой битвы

In [12]:
# Основной класс игры и запуск
class Game:
    """Основной класс игры"""
    def __init__(self):
        self.party: List[Character] = []
        self.boss: Optional[Boss] = None

    def create_party(self):
        """Создание партии персонажей"""
        print("🎮 СОЗДАНИЕ ПАРТИИ")

        classes = {
            "1": ("Воин", Warrior),
            "2": ("Маг", Mage),
            "3": ("Лекарь", Healer)
        }

        party_size = 3

        for i in range(party_size):
            print(f"\n🎭 Персонаж {i+1}:")
            print("1. ⚔️ Воин (сильный, много HP)")
            print("2. 🔮 Маг (интеллект, магические атаки)")
            print("3. 💚 Лекарь (лечение, поддержка)")

            while True:
                choice = input("Выберите класс: ")
                if choice in classes:
                    class_name, class_type = classes[choice]
                    name = input(f"Введите имя {class_name.lower()}: ")
                    character = class_type(name)
                    self.party.append(character)
                    print(f"✅ {class_name} {name} добавлен в партию!")
                    break
                else:
                    print("Неверный выбор!")

    def create_boss(self):
        """Создание босса"""
        difficulty = input("\n🎯 Выберите сложность (1-легкая, 2-нормальная, 3-сложная): ")
        level_multiplier = {"1": 0.8, "2": 1.0, "3": 1.2}
        multiplier = level_multiplier.get(difficulty, 1.0)

        boss_level = int(10 * multiplier)
        self.boss = Boss("Дракон Повелитель Тьмы", boss_level)
        print(f"🐉 Создан босс: {self.boss.name} (Уровень {boss_level})")

    def start_game(self):
        """Запуск игры"""
        print("🐉 ДОБРО ПОЖАЛОВАТЬ В МИНИ-ИГРУ!")
        print("Сразитесь с могучим боссом в пошаговом бою!")

        self.create_party()
        self.create_boss()

        seed = input("\n🌱 Введите seed для случайной генерации (или оставьте пустым): ")
        if seed:
            try:
                seed = int(seed)
            except ValueError:
                seed = hash(seed)
        else:
            seed = None

        battle = Battle(self.party, self.boss, seed)
        battle.run_battle()

# Запуск игры
if __name__ == "__main__":
    game = Game()
    game.start_game()

🐉 ДОБРО ПОЖАЛОВАТЬ В МИНИ-ИГРУ!
Сразитесь с могучим боссом в пошаговом бою!
🎮 СОЗДАНИЕ ПАРТИИ

🎭 Персонаж 1:
1. ⚔️ Воин (сильный, много HP)
2. 🔮 Маг (интеллект, магические атаки)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс: 1
Введите имя воин: федюк
✅ Воин федюк добавлен в партию!

🎭 Персонаж 2:
1. ⚔️ Воин (сильный, много HP)
2. 🔮 Маг (интеллект, магические атаки)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс: 2
Введите имя маг: элжей
✅ Маг элжей добавлен в партию!

🎭 Персонаж 3:
1. ⚔️ Воин (сильный, много HP)
2. 🔮 Маг (интеллект, магические атаки)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс: 3
Введите имя лекарь: кикоров
✅ Лекарь кикоров добавлен в партию!

🎯 Выберите сложность (1-легкая, 2-нормальная, 3-сложная): 2
🐉 Создан босс: Дракон Повелитель Тьмы (Уровень 10)

🌱 Введите seed для случайной генерации (или оставьте пустым): \
🎮 НАЧАЛО БИТВЫ!
👥 Партия: ['федюк', 'элжей', 'кикоров']
🐉 Босс: Дракон Повелитель Тьмы (Уровень 10)

🎯 РАУНД 1

--- Ход Дракон Повелитель Тьмы ---
